# Import library

In [17]:
import time
import os
from pprint import pprint
import pandas as pd
import torch
import sys

# for server_time using authentication
from pybit.usdt_perpetual import HTTP

# Helper function define

In [18]:
def whileTrue(func, **kwargs):
    err_end = False
    for _ in range(4):
        try:
            out = func(**kwargs)
            if out['ret_code'] == 0:
                if out['ext_code'] != "":
                    print("ext_code :", out['ext_code'])
                err_end = False
                break
            else:
                err_end = True
                print(
                    f"ret_code : {out['ret_code']}"
                    f"ret_msg : {out['ret_msg']}"
                    f"ext_code : {out['ext_code']}"
                    f"ext_info : {out['ext_info']}"
                )
                time.sleep(10)
        except:
            err_end = True
            print(f"Retry {func.__name__} ...")
            time.sleep(10)
    if err_end:
        out = None
    return out, err_end


# API ready

In [20]:
from bybit import api_key, api_secret

session = HTTP(
    endpoint='https://api.bybit.com',
    api_key=api_key,
    api_secret=api_secret,
)

# Hyperparameter

* Take Profit & Non-Liquidation (tp_cfdc, lqd_cfdc)
    * tp_cfdc: Take Profit probability
    * lqd_cfdc : Non-Liquidation probability
    * lower is more challenging, higher is more stable
    
||Take Profit probability (%)|Non-Liquidation probability (%)|
|-|-|-|
|0.00|50.000|50.000|
|0.25|59.871|40.129|
|0.30|61.791|38.209|
|0.40|65.542|34.458|
|0.45|67.364|32.636|
|0.50|69.146|30.854|
|0.75|77.337|22.663|
|1.00|84.134|15.866|
|1.50|93.319|6.681|
|2.00|97.725|2.275|
|2.50|99.379|0.621|
|3.00|99.865|0.135|
|3.50|99.977|0.023|
|4.50|100.000|0.000|


# Trade start

In [21]:
tp_cfdc, lqd_cfdc = 0.45, 2.0
output_dir = "./checkpoint/bybit_1"
data_path = "./data/bybit_data.pt"
candle_unit = 60
long_leverage_max, short_leverage_max = 100, 100
fee = 0.0006
mmr = 0.005
qty_unit = 0.001

In [22]:

result = torch.load(data_path)
open_long_time, open_short_time = 0, 0
buy_leverage, sell_leverage = 1, 1
pre_log, pre_load = 0, 0
long_cp, short_cp = {}, {}
long_cl, short_cl = {}, {}
init_margin, liquidation_time = {}, {}
old_order_time = 32400 + candle_unit*60*int(output_dir.split('_')[-1])
server_time = int(float(session.server_time()['time_now']))
while True:
    # --- Load Model --- #
    if int(time.time()) - pre_load > candle_unit * 60:
        load_dict = torch.load(output_dir + "/pytorch_model.bin", map_location='cpu')
        _ = model.load_state_dict(load_dict, False)
        del load_dict
        mean_std = torch.load("./mean_std.pt")
        high_mean, high_std = mean_std['high_mean'], mean_std['high_std']
        low_mean, low_std = mean_std['low_mean'], mean_std['low_std']
        print(
            f"\n"
            f"model reload complete!"
        )
        try:
            torch.save(result, data_path)
        except KeyboardInterrupt:
            torch.save(result, data_path)
            raise KeyboardInterrupt
        pre_load = int(time.time())
    
    qco = session.query_conditional_order(
        symbol="BTCUSDT",
    )

    # --- Erase Old Order --- #
    qco_no_erased = True
    for order in qco['result'][::-1]:
        if time.time() - time.mktime(time.strptime(order['created_time'][:-5], "%Y-%m-%dT%X")) > old_order_time:
            resp = session.cancel_conditional_order(
                symbol="BTCUSDT",
                stop_order_id=order['stop_order_id']
            )
            resp = session.place_active_order(
                symbol="BTCUSDT",
                side=order['side'],
                order_type="Market",
                qty=order['qty'],
                time_in_force="GoodTillCancel",
                reduce_only=True,
                close_on_trigger=True,
            )
            qco_no_erased = False
        else:
            break
    
    # --- Merge TP/SL Order --- #    
    if len(qco['result']) > 18 and qco_no_erased:
        brk = False
        for target in qco['result']:
            for order in qco['result'][::-1]:
                if order['side'] == target['side']:
                    resp = session.cancel_conditional_order(
                        symbol="BTCUSDT",
                        stop_order_id=order['stop_order_id'],
                    )
                    tp_size = order['qty'] + target['qty']
                    take_profit = order['qty'] * order['trigger_price'] + target['qty'] * target['trigger_price']
                    resp = session.replace_conditional_order(
                        symbol="BTCUSDT",
                        stop_order_id=target['stop_order_id'],
                        p_r_qty=int(tp_size * 1000 + .5) / 1000,
                        p_r_trigger_price=int(take_profit / tp_size * 2 + .5) / 2
                    )
                    brk = True
                    break
            if brk:
                break
    
    # --- Get PnL --- #
    ncpal = session.closed_profit_and_loss(
        symbol="BTCUSDT",
        start_time=server_time
    )

    if ncpal['result']['data'] is not None:
        for i in ncpal['result']['data']:
            if i['side'] == "Sell":
                if i['closed_pnl'] > 0:
                    long_cp[i['order_id']] = i['closed_pnl']
                else:
                    long_cl[i['order_id']] = i['closed_pnl']
            else:
                if i['closed_pnl'] > 0:
                    short_cp[i['order_id']] = i['closed_pnl']
                else:
                    short_cl[i['order_id']] = i['closed_pnl']
            if i['exec_type'] == 'BustTrade':
                liquidation_time[i['order_id']] = 1
            init_margin[i['order_id']] = i['cum_entry_value'] / i['leverage']

    # --- Logging --- #
    if int(time.time()) - pre_log > candle_unit * 5:
        total_trade_time = len(long_cp) + len(long_cl) + len(short_cp) + len(short_cl)
        long_pnl = sum(long_cp.values()) + sum(long_cl.values())
        short_pnl = sum(short_cp.values()) + sum(short_cl.values())
        lt_sum = sum(liquidation_time.values())
        im_sum = sum(init_margin.values())
        et = int(float(session.server_time()['time_now']) - server_time)
        days = et // 86400
        hour = et % 86400 // 3600
        mint = et % 86400 % 3600 // 60
        secs = et % 86400 % 3600 % 60
        print(
            f"\n\n"
            f"{time.strftime('%Y-%m-%d %T', time.localtime(time.time()))} ({days}days, {hour:02d}:{mint:02d}:{secs:02d} elapsed...)\n"
            f"Total = PnL : {long_pnl+short_pnl:.4f}$, win rate : {(len(long_cp) + len(short_cp)) / max(1, total_trade_time) * 100:.2f}%\n"
            f"Long = PnL : {long_pnl:.4f}$, win rate : {len(long_cp) / max(1, len(long_cp) + len(long_cl)) * 100:.2f}%\n"
            f"Short = PnL : {short_pnl:.4f}$, win rate : {len(short_cp) / max(1, len(short_cp) + len(short_cl)) * 100:.2f}%\n"
            f"Liquidation : {lt_sum/max(1, total_trade_time)*100:.2f}% ({lt_sum} / {total_trade_time})\n"
            f"Earn rate : {100 * (long_pnl+short_pnl) / max(1, im_sum):.2f}% ({im_sum+long_pnl+short_pnl:.4f}$ / {im_sum:.4f}$)"
        )
        pre_log = int(time.time())
    
    # --- Get Price --- #
    while True:
        nowtime = int(time.time())
        i = 0
        while True:
            i += 1
            resp = session.query_kline(
                symbol="BTCUSDT",
                interval="1",
                limit=200,
                from_time=(nowtime//60 - 200*i + 1) * 60
            )

            df = resp['result']
            df = pd.DataFrame(df)
            dfl = df[['open', 'high', 'low', 'close', 'volume']].to_numpy().tolist()
            df = {k: v for k,v in zip(df['id'], dfl)}
            before_len = len(result)
            result.update(df)
            if before_len == len(result):
                break
        if i == 1:
            break

    rst = sorted(result.items(), key=lambda x:x[0])
    rst = [i[1] for i in rst[-1024*candle_unit-1:-1]]
    rst = torch.tensor(rst)

    sliced = rst[:, :-1].reshape(1024, -1)
    opening = sliced[:, 0]
    high, _ = sliced.max(dim=-1)
    low, _ = sliced.min(dim=-1)
    close = sliced[:, -1]
    sliced = rst[:, -1].reshape(1024, -1)
    volume = sliced.sum(dim=-1, keepdim=True)
    enc = torch.stack((opening, high, low, close), dim=-1)
    enc = torch.cat((enc, volume), dim=-1)

    _, outputs = model(inputs_embeds=enc)
    pred_high, pred_low = outputs[-1]
    pred_high = pred_high.item()
    pred_low = pred_low.item()

    # --- Order --- #
    gwb = session.get_wallet_balance(coin="USDT")['result']['USDT']
    if gwb['available_balance'] > gwb['used_margin']:
        lifs = session.latest_information_for_symbol(
            symbol="BTCUSDT"
        )

        if float(session.server_time()['time_now']) > open_long_time:
            if pred_high / (high_mean + high_std*tp_cfdc) > float(lifs['result'][0]['ask_price']) * (1+fee)/(1-fee):
                resp, err_end = whileTrue(session.place_active_order,
                    symbol="BTCUSDT",
                    side="Buy",
                    order_type="Market",
                    qty=qty_unit,
                    time_in_force="GoodTillCancel",
                    reduce_only=False,
                    close_on_trigger=False,
                    take_profit=int(pred_high * 2 / (high_mean + high_std*tp_cfdc) + .5)/2,
                )
                open_long_time = float(session.server_time()['time_now']) // 60 * 60 + 60
        
        if float(session.server_time()['time_now']) > open_short_time:
            if pred_low / (low_mean - low_std*tp_cfdc) < float(lifs['result'][0]['bid_price']) * (1-fee)/(1+fee):
                resp, err_end = whileTrue(session.place_active_order,
                    symbol="BTCUSDT",
                    side="Sell",
                    order_type="Market",
                    qty=qty_unit,
                    time_in_force="GoodTillCancel",
                    reduce_only=False,
                    close_on_trigger=False,
                    take_profit=int(pred_low * 2 / (low_mean - low_std*tp_cfdc) + .5)/2,
                )
                open_short_time = float(session.server_time()['time_now']) // 60 * 60 + 60

    # --- Set Leverage --- #
    my_position = session.my_position(
        symbol="BTCUSDT"
    )
    need_order = {}
    for mp in my_position['result']:
        side = mp.pop('side')
        need_order[side] = mp
    my_position = need_order
    
    if my_position['Buy']['entry_price'] > 0:
        buy_leverage = min(long_leverage_max, max(1, 1 / (1 + mmr - pred_low / (low_mean + low_std*lqd_cfdc) / my_position['Buy']['entry_price'])))
    if my_position['Sell']['entry_price'] > 0:
        sell_leverage = min(short_leverage_max, max(1, 1 / (pred_high / (high_mean - high_std*lqd_cfdc) / my_position['Sell']['entry_price'] - 1 + mmr)))

    if abs(my_position['Buy']['leverage'] - buy_leverage) > 0.01 or abs(my_position['Sell']['leverage'] - sell_leverage) > 0.01:
        set_leverage = session.set_leverage(
            symbol="BTCUSDT",
            buy_leverage=buy_leverage,
            sell_leverage=sell_leverage,
        )



model reload complete!


2022-10-16 16:12:59 (0days, 00:00:07 elapsed...)
Total = PnL : 0.0000$, win rate : 0.00%
Long = PnL : 0.0000$, win rate : 0.00%
Short = PnL : 0.0000$, win rate : 0.00%
Liquidation : 0.00% (0 / 0)
Earn rate : 0.00% (0.0000$ / 0.0000$)


2022-10-16 16:18:01 (0days, 00:05:09 elapsed...)
Total = PnL : 0.0000$, win rate : 0.00%
Long = PnL : 0.0000$, win rate : 0.00%
Short = PnL : 0.0000$, win rate : 0.00%
Liquidation : 0.00% (0 / 0)
Earn rate : 0.00% (0.0000$ / 0.0000$)


2022-10-16 16:23:03 (0days, 00:10:10 elapsed...)
Total = PnL : 0.0000$, win rate : 0.00%
Long = PnL : 0.0000$, win rate : 0.00%
Short = PnL : 0.0000$, win rate : 0.00%
Liquidation : 0.00% (0 / 0)
Earn rate : 0.00% (0.0000$ / 0.0000$)


2022-10-16 16:28:05 (0days, 00:15:12 elapsed...)
Total = PnL : 0.0000$, win rate : 0.00%
Long = PnL : 0.0000$, win rate : 0.00%
Short = PnL : 0.0000$, win rate : 0.00%
Liquidation : 0.00% (0 / 0)
Earn rate : 0.00% (0.0000$ / 0.0000$)


2022-10-16 16:33:06 (0days, 00